In [24]:
import argparse
import time
import itertools

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

class AutoDictionary(dict):
    def __getitem__(self, item):
        try:
            return dict.__getitem__(self, item)
        except KeyError:
            value = self[item] = type(self)()
            return value

name  = "data_"
nodes = ["5653", "55ad", "55e4", "5599", "55dd", "5565", "560b", "5632", "55b3", "5563", "630a"]
#nodes = ["5599"]
ext = '.csv'

names  = ["FSK", "OQPSK", "OFDM"] 
values = ["fsk_rssi", "oqpsk_rssi", "ofdm_rssi"]

packets_per_hour = 60
total_rx_packets = 0

result = AutoDictionary()

# Iterate over all nodes
for node in nodes:
    print("Parsing node = {} ".format(node))
    
    # Read values from csv
    filename = name + node + ext
    data = pd.read_csv(filename, index_col=0) 
    
    # Create date column based on day and hour
    data['date'] = data['day'] + ' ' + data['hour']
    data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d %H:%M:%S')
    
    # Drop unnecessary columns
    data = data.drop(['day','hour'], axis=1)
    
    # Set date as index
    data = data.set_index(["date"])
    
    # Repeat for all modulations (i.e., FSK, OQPSK, OFDM)
    for v in values:    
        # Count packets received hourly for each retry
        x1_pdr  = data[data['pkt_retry'] == 0].groupby(pd.Grouper(freq='H', level='date'))[v].count()
        x1_rssi = data[data['pkt_retry'] == 0].groupby(pd.Grouper(freq='H', level='date'))[v].mean()
        x2_pdr  = data[data['pkt_retry'] == 1].groupby(pd.Grouper(freq='H', level='date'))[v].count()
        x2_rssi = data[data['pkt_retry'] == 1].groupby(pd.Grouper(freq='H', level='date'))[v].mean()
        x3_pdr  = data[data['pkt_retry'] == 2].groupby(pd.Grouper(freq='H', level='date'))[v].count()
        x3_rssi = data[data['pkt_retry'] == 2].groupby(pd.Grouper(freq='H', level='date'))[v].mean()
        
        x1_pdr = (100 * x1_pdr / packets_per_hour).clip(0, 100)
        x2_pdr = (100 * x2_pdr / packets_per_hour).clip(0, 100)
        x3_pdr = (100 * x3_pdr / packets_per_hour).clip(0, 100)
        
        result[node][v]['rssi'] = [x1_rssi, x2_rssi, x3_rssi]
        result[node][v]['pdr']  = [x1_pdr, x2_pdr, x3_pdr]

Parsing node = 5653 
Parsing node = 55ad 
Parsing node = 55e4 
Parsing node = 5599 
Parsing node = 55dd 
Parsing node = 5565 
Parsing node = 560b 
Parsing node = 5632 
Parsing node = 55b3 
Parsing node = 5563 
Parsing node = 630a 


In [ ]:
nrow = 6
ncol = 2

fig, axs = plt.subplots(nrow, ncol, figsize=(20,20), constrained_layout=True)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

for ax, key in zip(enumerate(fig.axes), result.keys()):
    # Recover data
    fsk_rssi   = result[key]["fsk_rssi"]['rssi']
    fsk_pdr    = result[key]["fsk_rssi"]['pdr']
    oqpsk_rssi = result[key]["oqpsk_rssi"]['rssi']
    oqpsk_pdr  = result[key]["oqpsk_rssi"]['pdr']
    ofdm_rssi  = result[key]["ofdm_rssi"]['rssi']
    ofdm_pdr   = result[key]["ofdm_rssi"]['pdr']
    
    # Plot FSK
    ax[1].scatter(fsk_rssi[0], fsk_pdr[0], c=colors[0], s=0.35, label="FSK")
    ax[1].scatter(fsk_rssi[1], fsk_pdr[1], c=colors[0], s=0.35)
    ax[1].scatter(fsk_rssi[2], fsk_pdr[2], c=colors[0], s=0.35)
    
    # Plot OQPSK
    ax[1].scatter(oqpsk_rssi[0], oqpsk_pdr[0], c=colors[1], s=0.35, label="OQPSK")
    ax[1].scatter(oqpsk_rssi[1], oqpsk_pdr[1], c=colors[1], s=0.35)
    ax[1].scatter(oqpsk_rssi[2], oqpsk_pdr[2], c=colors[1], s=0.35)
    
    # Plot OFDM
    ax[1].scatter(ofdm_rssi[0], ofdm_pdr[0], c=colors[2], s=0.35, label="OFDM")
    ax[1].scatter(ofdm_rssi[1], ofdm_pdr[1], c=colors[2], s=0.35)
    ax[1].scatter(ofdm_rssi[2], ofdm_pdr[2], c=colors[2], s=0.35)
    
    # Configure plot
    ax[1].legend()
    name = (key[:2] + '-' + key[2:]).upper()
    ax[1].set_title("EUI-64 = {}".format(name))
    ax[1].grid()
    ax[1].set_ylim(0, 100)
    ax[1].set_xlim(-120, -60)
    ax[1].set_ylabel("PDR (%)")
    
    # Change the marker size manually
    lgnd = ax[1].legend()
    lgnd.legendHandles[0]._sizes = [20]
    lgnd.legendHandles[1]._sizes = [20]
    lgnd.legendHandles[2]._sizes = [20]

fig.delaxes(axs[5][1])
fig.suptitle("PDR vs. RSSI")
plt.savefig("05_rssi_pdr.png", dpi=300)